# exp080
derivedデータを使ったbaselineの復元  
https://www.kaggle.com/hyperc/gsdc-reproducing-baseline-wls-on-one-measurement

In [47]:
# import library
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib_venn import venn2, venn2_circles
import seaborn as sns
from tqdm.notebook import tqdm
import pathlib
import plotly
import plotly.express as px
import itertools
import lightgbm as lgb
from optuna.integration import lightgbm as optuna_lgb
import simdkalman
import optuna
import pyproj
from pyproj import Proj, transform
from sklearn import metrics
from sklearn.metrics import roc_curve, precision_recall_curve, confusion_matrix, accuracy_score
pd.set_option('display.max_rows', 100)
from math import * 
import scipy.optimize as opt

In [2]:
import ipynb_path

def get_nb_name():
    nb_path = ipynb_path.get()
    nb_name = nb_path.rsplit('/',1)[1].replace('.ipynb','')
    return nb_name

In [3]:
# directory setting
nb_name = get_nb_name()
INPUT = '../input/google-smartphone-decimeter-challenge'
OUTPUT = '../output/' + nb_name
os.makedirs(OUTPUT, exist_ok=True)

# utils

In [4]:
def get_train_score(df, gt):
    gt = gt.rename(columns={'latDeg':'latDeg_gt', 'lngDeg':'lngDeg_gt'})
    df = df.merge(gt, on=['collectionName', 'phoneName', 'millisSinceGpsEpoch'], how='inner')
    # calc_distance_error
    df['err'] = calc_haversine(df['latDeg_gt'], df['lngDeg_gt'], df['latDeg'], df['lngDeg'])
    # calc_evaluate_score
    df['phone'] = df['collectionName'] + '_' + df['phoneName']
    res = df.groupby('phone')['err'].agg([percentile50, percentile95])
    res['p50_p90_mean'] = (res['percentile50'] + res['percentile95']) / 2 
    score = res['p50_p90_mean'].mean()
    return score

In [5]:
def calc_haversine(lat1, lon1, lat2, lon2):
    """Calculates the great circle distance between two points
    on the earth. Inputs are array-like and specified in decimal degrees.
    """
    RADIUS = 6_367_000
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + \
        np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    dist = 2 * RADIUS * np.arcsin(a**0.5)
    return dist

In [6]:
def visualize_trafic(df, center, zoom=9):
    fig = px.scatter_mapbox(df,
                            
                            # Here, plotly gets, (x,y) coordinates
                            lat="latDeg",
                            lon="lngDeg",
                            
                            #Here, plotly detects color of series
                            color="phoneName",
                            labels="phoneName",
                            
                            zoom=zoom,
                            center=center,
                            height=600,
                            width=800)
    fig.update_layout(mapbox_style='stamen-terrain')
    fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
    fig.update_layout(title_text="GPS trafic")
    fig.show()
    
def visualize_collection(df, collection):
    target_df = df[df['collectionName']==collection].copy()
    lat_center = target_df['latDeg'].mean()
    lng_center = target_df['lngDeg'].mean()
    center = {"lat":lat_center, "lon":lng_center}
    
    visualize_trafic(target_df, center)

In [7]:
# ground_truth
def get_ground_truth():
    p = pathlib.Path(INPUT)
    gt_files = list(p.glob('train/*/*/ground_truth.csv'))

    gts = []
    for gt_file in gt_files:
        gts.append(pd.read_csv(gt_file))
    ground_truth = pd.concat(gts)

    return ground_truth

In [8]:
def percentile50(x):
    return np.percentile(x, 50)
def percentile95(x):
    return np.percentile(x, 95)

In [9]:
class train_result:
    def __init__(self, df):
        self.df = df
        self.gt = get_ground_truth()
        self.bl = pd.read_csv(INPUT + '/' + 'baseline_locations_train.csv')
        
        self.gt = self.gt.rename(columns={'latDeg':'latDeg_gt', 'lngDeg':'lngDeg_gt'})
        self.df = self.df.merge(self.gt, on=['collectionName', 'phoneName', 'millisSinceGpsEpoch'], how='inner')
        self.df['phone'] = self.df['collectionName'] + '_' + self.df['phoneName']
        self.df['err'] =  calc_haversine(self.df['latDeg_gt'], self.df['lngDeg_gt'], self.df['latDeg'], self.df['lngDeg'])
        
        self.phone_res = self.calc_err('phone')
        self.clc_res = self.calc_err('collectionName')
        self.phonename_res = self.calc_err('phoneName')
        
    def calc_err(self, by):
        res = self.df.groupby(by)['err'].agg([percentile50, percentile95])
        res['p50_p90_mean'] = (res['percentile50'] + res['percentile95']) / 2
        return res
    
    @property
    def score(self):
        return self.phone_res['p50_p90_mean'].mean()
    @property
    def raw_data(self):
        return self.df
    @property
    def err(self):
        return self.phone_res
    @property
    def collection_err(self):
        return self.clc_res
    @property
    def phonename_err(self):
        return self.phonename_res
    
    def viz_map(self, collection, show_gt=True, show_bl=True):
        tmp = self.df[self.df['collectionName']==collection][['collectionName', 'phoneName', 'latDeg', 'lngDeg']]
        tmp2 = self.df[self.df['collectionName']==collection][['collectionName', 'phoneName', 'latDeg_gt', 'lngDeg_gt']]
        tmp2 = tmp2.rename(columns={'latDeg_gt':'latDeg', 'lngDeg_gt':'lngDeg'})
        tmp2['phoneName'] = tmp2['phoneName'] + '_GT'
        tmp3 = self.bl[self.bl['collectionName']==collection][['collectionName', 'phoneName', 'latDeg', 'lngDeg']]
        tmp3['phoneName'] = tmp3['phoneName'] + '_BL'
        
        if show_gt:
            tmp = tmp.append(tmp2)
        if show_bl:
            tmp = tmp.append(tmp3)
        visualize_collection(tmp, collection)

In [10]:
def get_data():
    base_train = pd.read_csv(INPUT + '/' + 'baseline_locations_train.csv')
    base_test = pd.read_csv(INPUT + '/' + 'baseline_locations_test.csv')
    sample_sub = pd.read_csv(INPUT + '/' + 'sample_submission.csv')
    ground_truth = pd.read_csv(INPUT + '/prep/ground_truth_train.csv')
    return base_train, base_test, sample_sub, ground_truth

In [11]:
def ecef2lla(x, y, z):
    # x, y and z are scalars or vectors in meters
    x = np.array([x]).reshape(np.array([x]).shape[-1], 1)
    y = np.array([y]).reshape(np.array([y]).shape[-1], 1)
    z = np.array([z]).reshape(np.array([z]).shape[-1], 1)

    a=6378137
    a_sq=a**2
    e = 8.181919084261345e-2
    e_sq = 6.69437999014e-3

    f = 1/298.257223563
    b = a*(1-f)

    # calculations:
    r = np.sqrt(x**2 + y**2)
    ep_sq  = (a**2-b**2)/b**2
    ee = (a**2-b**2)
    f = (54*b**2)*(z**2)
    g = r**2 + (1 - e_sq)*(z**2) - e_sq*ee*2
    c = (e_sq**2)*f*r**2/(g**3)
    s = (1 + c + np.sqrt(c**2 + 2*c))**(1/3.)
    p = f/(3.*(g**2)*(s + (1./s) + 1)**2)
    q = np.sqrt(1 + 2*p*e_sq**2)
    r_0 = -(p*e_sq*r)/(1+q) + np.sqrt(0.5*(a**2)*(1+(1./q)) - p*(z**2)*(1-e_sq)/(q*(1+q)) - 0.5*p*(r**2))
    u = np.sqrt((r - e_sq*r_0)**2 + z**2)
    v = np.sqrt((r - e_sq*r_0)**2 + (1 - e_sq)*z**2)
    z_0 = (b**2)*z/(a*v)
    h = u*(1 - b**2/(a*v))
    phi = np.arctan((z + ep_sq*z_0)/r)
    lambd = np.arctan2(y, x)

    return phi*180/np.pi, lambd*180/np.pi, h

# データ取得

In [13]:
train, test, sub, gt = get_data()
train_derived = pd.read_csv(INPUT + '/prep/derived_train.csv')

train_derived['phone'] = train_derived['collectionName'] + '_' + train_derived['phoneName']

# 時間差の確認

In [14]:
train_grouped = train.groupby('phone')['millisSinceGpsEpoch'].agg(['min', 'max']).reset_index()
train_grouped.columns = ['phone', 'train_min', 'train_max']

derived_grouped = train_derived.groupby('phone')['millisSinceGpsEpoch'].agg(['min', 'max']).reset_index()
derived_grouped.columns = ['phone', 'derived_min', 'derived_max']

In [15]:
result = train_grouped.merge(derived_grouped, on='phone', how='left')

In [16]:
result['min_diff'] = result['derived_min'] - result['train_min']
result['max_diff'] = result['derived_max'] - result['train_max']

In [20]:
result.to_csv(OUTPUT + '/timediff.csv')
result

,phone,train_min,train_max,derived_min,derived_max,min_diff,max_diff
0,2020-05-14-US-MTV-1_Pixel4,1273529463442,1273531202442,1273529464442,1273531202442,1000,0
1,2020-05-14-US-MTV-1_Pixel4XLModded,1273529466449,1273531211449,1273529467448,1273531211448,999,-1
2,2020-05-14-US-MTV-2_Pixel4,1273538830442,1273540599442,1273538831442,1273540599442,1000,0
3,2020-05-14-US-MTV-2_Pixel4XLModded,1273538835449,1273539436656,1273538836448,1273539436655,999,-1
4,2020-05-21-US-MTV-1_Pixel4,1274119793431,1274121827431,1274119794431,1274121827431,1000,0
5,2020-05-21-US-MTV-2_Pixel4,1274131364434,1274133328434,1274131365434,1274133328434,1000,0
6,2020-05-21-US-MTV-2_Pixel4XL,1274131361444,1274133154444,1274131362443,1274133154443,999,-1
7,2020-05-29-US-MTV-1_Pixel4,1274827486438,1274829398439,1274827487438,1274829398438,1000,-1
8,2020-05-29-US-MTV-1_Pixel4XL,1274827488443,1274829399443,1274827489442,1274829399442,999,-1
9,2020-05-29-US-MTV-1_Pixel4XLModded,1274827485440,1274829410440,1274827486440,1274829410440,1000,0


In [21]:
tmp1 = train[['phone', 'millisSinceGpsEpoch']].copy()
tmp1['train'] = 1

tmp2 = train_derived[['phone', 'millisSinceGpsEpoch']].copy()
tmp2 = tmp2.drop_duplicates()
tmp2['derived'] = 1

tmp = tmp1.merge(tmp2, on=['phone', 'millisSinceGpsEpoch'], how='outer')
tmp = tmp.sort_values(['phone', 'millisSinceGpsEpoch'])

In [23]:
tmp.to_csv(OUTPUT + '/time_compare.csv')
tmp

,phone,millisSinceGpsEpoch,train,derived
0,2020-05-14-US-MTV-1_Pixel4,1273529463442,1.0,NaN
1,2020-05-14-US-MTV-1_Pixel4,1273529464442,1.0,1.0
2,2020-05-14-US-MTV-1_Pixel4,1273529465442,1.0,1.0
3,2020-05-14-US-MTV-1_Pixel4,1273529466442,1.0,1.0
4,2020-05-14-US-MTV-1_Pixel4,1273529467442,1.0,1.0
...,...,...,...,...
131337,2021-04-29-US-SJC-2_SamsungS20Ultra,1303760315000,1.0,1.0
131338,2021-04-29-US-SJC-2_SamsungS20Ultra,1303760316000,1.0,1.0
131339,2021-04-29-US-SJC-2_SamsungS20Ultra,1303760317000,1.0,1.0
131340,2021-04-29-US-SJC-2_SamsungS20Ultra,1303760318000,1.0,1.0


In [29]:
tmp['s_diff'] = tmp['millisSinceGpsEpoch'] - tmp['millisSinceGpsEpoch'].shift(1)
tmp.loc[tmp['phone']!=tmp['phone'].shift(1)] = np.nan

In [35]:
tmp.groupby(['phone', 's_diff'])['millisSinceGpsEpoch'].count().reset_index().to_csv(OUTPUT + '/time_compare2.csv')

In [39]:
pd.DataFrame(tmp.s_diff.value_counts()).head(100)

,s_diff
1000.0,68974
1.0,54228
999.0,46144
991.0,1210
992.0,1126
990.0,966
993.0,875
1011.0,766
1012.0,729
1013.0,669


# baselineの再作成

In [99]:
def prepare_calc_baseline(df):
    light_speed = 299_792_458
    omega_e = 7.2921151467e-5
    
    # Corrected pseudorange according to data instructions
    df['correctedPrM'] = df['rawPrM'] + \
                         df['satClkBiasM'] - \
                         df['isrbM'] - \
                         df['ionoDelayM'] - \
                         df['tropoDelayM']
    
    # Time it took for signal to travel
    df['transmissionTimeSeconds'] = df['correctedPrM'] / light_speed
    
    # Compute true sat positions at arrival time
    df['xSatPosMRotated'] = \
        np.cos(omega_e * df['transmissionTimeSeconds']) * df['xSatPosM'] \
        + np.sin(omega_e * df['transmissionTimeSeconds']) * df['ySatPosM']

    df['ySatPosMRotated'] = \
        - np.sin(omega_e * df['transmissionTimeSeconds']) * df['xSatPosM'] \
        + np.cos(omega_e * df['transmissionTimeSeconds']) * df['ySatPosM']

    df['zSatPosMRotated'] = df['zSatPosM']
    
    # Uncertainty weight for the WLS method
    df['uncertaintyWeight'] = 1 / df['rawPrUncM']
    
    return df

In [100]:
def calc_baseline_point(df):

    def distance(sat_pos, x):
        sat_pos_diff = sat_pos.copy(deep=True)

        sat_pos_diff['xSatPosMRotated'] = sat_pos_diff['xSatPosMRotated'] - x[0]
        sat_pos_diff['ySatPosMRotated'] = sat_pos_diff['ySatPosMRotated'] - x[1]
        sat_pos_diff['zSatPosMRotated'] = sat_pos_diff['zSatPosMRotated'] - x[2]

        sat_pos_diff['d'] = sat_pos_diff['uncertaintyWeight'] * \
                            (np.sqrt((sat_pos_diff['xSatPosMRotated']**2 + sat_pos_diff['ySatPosMRotated']**2 + sat_pos_diff['zSatPosMRotated']**2)) + \
                             x[3] - sat_pos_diff['correctedPrM'])

        return sat_pos_diff['d']

    def distance_fixed_satpos(x):
        return distance(df[['xSatPosMRotated', 'ySatPosMRotated', 'zSatPosMRotated', 'correctedPrM', 'uncertaintyWeight']], x)
    
    x0 = [0,0,0,0]
    opt_res = opt.least_squares(distance_fixed_satpos, x0)
    # Optimiser yields a position in the ECEF coordinates
    opt_res_pos = opt_res.x
    
    # ECEF position to lat/long
    wls_estimated_pos = ecef2lla(*opt_res_pos[:3])
    wls_estimated_pos = np.squeeze(wls_estimated_pos)
    
    return wls_estimated_pos[0], wls_estimated_pos[1]

In [101]:
def calc_baseline1(df, derived):
    idx = list(df.index)
    phone_list = []
    s_list = []
    n_list = []
    lat_list = []
    lng_list = []
    
    for i in tqdm(idx):
        phone = df.at[i, 'phone']
        s = df.at[i, 'millisSinceGpsEpoch']
        tmp = derived[(derived['phone']==phone) & (derived['millisSinceGpsEpoch']==s)].copy()
        n = len(tmp)
        phone_list.append(phone)
        s_list.append(s)
        n_list.append(n)
        
        if n == 0:    
            lat_list.append(np.nan)
            lng_list.append(np.nan)
        
        else:
            res = calc_baseline_point(tmp)
            lat_list.append(res[0])
            lng_list.append(res[1])
    
    output_df = pd.DataFrame()
    output_df['phone'] = phone_list
    output_df['millisSinceGpsEpoch'] = s_list
    output_df['latDeg'] = lat_list
    output_df['lngDeg'] = lng_list
    output_df['n'] = n_list
    
    return output_df

In [102]:
def calc_baseline2(df, derived, s_th):
    idx = list(df.index)
    phone_list = []
    s_list = []
    n_list = []
    lat_list = []
    lng_list = []
    
    for i in tqdm(idx):
        phone = df.at[i, 'phone']
        s = df.at[i, 'millisSinceGpsEpoch'] + 1000
        s_lower = s - s_th
        s_upper = s + s_th
        tmp = derived[(derived['phone']==phone) & (derived['millisSinceGpsEpoch']>=s_lower) & (derived['millisSinceGpsEpoch']<=s_upper)].copy()
        n = len(tmp)
        phone_list.append(phone)
        s_list.append(s)
        n_list.append(n)
        
        if n == 0:    
            lat_list.append(np.nan)
            lng_list.append(np.nan)
        
        else:
            res = calc_baseline_point(tmp)
            lat_list.append(res[0])
            lng_list.append(res[1])
    
    output_df = pd.DataFrame()
    output_df['phone'] = phone_list
    output_df['millisSinceGpsEpoch'] = s_list
    output_df['latDeg'] = lat_list
    output_df['lngDeg'] = lng_list
    output_df['n'] = n_list
    
    return output_df

In [103]:
train, test, sub, gt = get_data()
train_derived = pd.read_csv(INPUT + '/prep/derived_train.csv')
train_derived['phone'] = train_derived['collectionName'] + '_' + train_derived['phoneName']

In [104]:
train_derived = prepare_calc_baseline(train_derived)

In [105]:
train_target = train[train['phone']=='2020-05-14-US-MTV-1_Pixel4'].copy()

In [106]:
result1 = calc_baseline1(train_target, train_derived)

  0%|          | 0/1740 [00:00<?, ?it/s]

In [110]:
result2 = calc_baseline2(train_target, train_derived, 5)

  0%|          | 0/1740 [00:00<?, ?it/s]

In [124]:
result1['collectionName'] = '2020-05-14-US-MTV-1'
result1['phoneName'] = 'Pixel4'

result2['collectionName'] = '2020-05-14-US-MTV-1'
result2['phoneName'] = 'Pixel4'

In [118]:
gt['phone'] = gt['collectionName'] + '_' + gt['phoneName']

In [119]:
gt_target = gt[gt['phone']=='2020-05-14-US-MTV-1_Pixel4'].copy()

In [126]:
result = train_target.copy()
result[['lat_gt', 'lng_gt']] = gt_target[['latDeg', 'lngDeg']]
result[['lat_1', 'lng_1']] = result1[['latDeg', 'lngDeg']]
result[['lat_2', 'lng_2']] = result2[['latDeg', 'lngDeg']]
result = result.dropna()

In [131]:
idx = result.index
print(get_train_score(train_target.loc[idx], gt_target.loc[idx]))
print(get_train_score(result1.loc[idx], gt_target.loc[idx]))
print(get_train_score(result2.loc[idx], gt_target.loc[idx]))

2.0969359976311575
27.170097416490353
27.172556103381567


In [136]:
result['baseline_err'] = calc_haversine(result['lat_gt'], result['lng_gt'], result['latDeg'], result['lngDeg'])
result['res1_err'] = calc_haversine(result['lat_gt'], result['lng_gt'], result['lat_1'], result['lng_1'])
result['res2_err'] = calc_haversine(result['lat_gt'], result['lng_gt'], result['lat_2'], result['lng_2'])

In [137]:
result.to_csv(OUTPUT + '/result.csv')

In [139]:
result

,collectionName,phoneName,millisSinceGpsEpoch,latDeg,lngDeg,heightAboveWgs84EllipsoidM,phone,lat_gt,lng_gt,lat_1,lng_1,lat_2,lng_2,baseline_err,res1_err,res2_err
1,2020-05-14-US-MTV-1,Pixel4,1273529464442,37.423578,-122.094101,-33.29,2020-05-14-US-MTV-1_Pixel4,37.423576,-122.094132,37.423579,-122.094092,37.423577,-122.094099,2.745901,3.574413,2.893043
2,2020-05-14-US-MTV-1,Pixel4,1273529465442,37.423573,-122.094111,-30.99,2020-05-14-US-MTV-1_Pixel4,37.423576,-122.094132,37.423577,-122.094099,37.423574,-122.094113,1.888409,2.892709,1.703368
3,2020-05-14-US-MTV-1,Pixel4,1273529466442,37.423583,-122.094121,-32.83,2020-05-14-US-MTV-1_Pixel4,37.423576,-122.094132,37.423574,-122.094113,37.423578,-122.094120,1.213483,1.703400,1.134983
4,2020-05-14-US-MTV-1,Pixel4,1273529467442,37.423579,-122.094114,-34.49,2020-05-14-US-MTV-1_Pixel4,37.423576,-122.094132,37.423578,-122.094120,37.423575,-122.094118,1.650722,1.134854,1.261353
5,2020-05-14-US-MTV-1,Pixel4,1273529468442,37.423578,-122.094126,-33.57,2020-05-14-US-MTV-1_Pixel4,37.423576,-122.094132,37.423575,-122.094118,37.423572,-122.094120,0.557491,1.261785,1.148691
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1734,2020-05-14-US-MTV-1,Pixel4,1273531197442,37.645815,-122.405605,-34.35,2020-05-14-US-MTV-1_Pixel4,37.645823,-122.405608,37.645807,-122.405601,37.645807,-122.405601,0.947812,1.888099,1.926158
1735,2020-05-14-US-MTV-1,Pixel4,1273531198442,37.645814,-122.405615,-33.66,2020-05-14-US-MTV-1_Pixel4,37.645823,-122.405608,37.645807,-122.405601,37.645814,-122.405615,1.165287,1.926188,1.179629
1736,2020-05-14-US-MTV-1,Pixel4,1273531199442,37.645819,-122.405615,-34.77,2020-05-14-US-MTV-1_Pixel4,37.645823,-122.405608,37.645814,-122.405615,37.645812,-122.405609,0.763136,1.179706,1.227293
1737,2020-05-14-US-MTV-1,Pixel4,1273531200442,37.645814,-122.405599,-35.89,2020-05-14-US-MTV-1_Pixel4,37.645823,-122.405608,37.645812,-122.405609,37.645805,-122.405593,1.338661,1.227286,2.454867


In [141]:
score0 = result.groupby('phone')['baseline_err'].agg([percentile50, percentile95])
score0['p50_p90_mean'] = (score0['percentile50'] + score0['percentile95']) / 2 
print(score0['p50_p90_mean'].mean())

2.0969359976311575


In [142]:
score0 = result.groupby('phone')['res1_err'].agg([percentile50, percentile95])
score0['p50_p90_mean'] = (score0['percentile50'] + score0['percentile95']) / 2 
print(score0['p50_p90_mean'].mean())

27.170097416490353


In [143]:
score0 = result.groupby('phone')['res2_err'].agg([percentile50, percentile95])
score0['p50_p90_mean'] = (score0['percentile50'] + score0['percentile95']) / 2 
print(score0['p50_p90_mean'].mean())

2.1643485543233707
